In [ ]:
import os

os.environ['AZURE_SEARCH_SERVICE_ENDPOINT'] = 'https://chatwithdata505.search.windows.net'
os.environ['AZURE_SEARCH_API_KEY'] = ''

os.environ['OPENAI_API_KEY'] = ''
os.environ['OPENAI_API_BASE'] = 'https://aoai-gpt4-505.openai.azure.com/'


In [ ]:
import os

import autogen
from autogen import OpenAIWrapper
from autogen import AssistantAgent, UserProxyAgent


from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt


In [ ]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizedQuery

search_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = 'ifrs9'
search_api_key = os.environ["AZURE_SEARCH_API_KEY"]

import openai

openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

In [ ]:

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embeddings(text: str):
    # There are a few ways to get embeddings. This is just one example.
    import openai

    open_ai_endpoint = os.getenv("OPENAI_API_BASE")
    open_ai_key = os.getenv("OPENAI_API_KEY")

    client = openai.AzureOpenAI(
        azure_endpoint=open_ai_endpoint,
        api_key=open_ai_key,
        api_version="2023-09-01-preview",
    )
    embedding = client.embeddings.create(input=[text], model="text-embedding-ada-002")
    return embedding.data[0].embedding


In [ ]:
def simple_hybrid_search(query, fields="contentVector"):

    search_client = SearchClient(search_endpoint, index_name, AzureKeyCredential(search_api_key))
    vector_query = VectorizedQuery(vector=get_embeddings(query), k_nearest_neighbors=3, fields=fields)

    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        top=10,
        select=["content", "id"],
    )

    for result in results:
        print(f"{result['content']}, \n score: {result['@search.score']}")
        # print(result)
        
simple_hybrid_search(query="what is sppi?")

In [ ]:
open_ai_endpoint = os.getenv("OPENAI_API_BASE")
open_ai_key = os.getenv("OPENAI_API_KEY")
model_name = "gpt-4"

client = openai.AzureOpenAI(
    azure_endpoint=open_ai_endpoint,
    api_key=open_ai_key,
    api_version="2023-09-01-preview",
)

# messages = [{"role": "user", "content": "what is sppi?"}]

# messages = [{"role": "user", "content": "Help me find a good mexican recipe that has beans and rice"}]
# messages = [{"role": "user", "content": "what is sppi?"}]

### Try again with a more detailed system message ###
system_message = """Assistant is a large language model designed to help users find answer to his question about Classification of financial instruments under IFRS 9.
You have access to an Azure Cognitive Search index with all book about Classification of financial instruments under IFRS 9.
You are designed to be an interactive assistant, so you can ask users clarifying questions to help them find the right recipe. It's better to give more detailed queries to the search index rather than vague one.
"""

messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": "what is sppi?"}]

# messages = [{"role": "system", "content": system_message},
#            {"role": "user", "content": "find an easy mexican recipe with beans and rice"}]

tools = [
    {
        "type": "function",
        "function": {
            "name": "query_recipes",
            "description": "Retrieve context from the Azure Cognitive Search index",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query string to search for context",
                    }
                },
                "required": ["query"],
            },
        },
    }
]

response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    tools=tools,
    temperature=0.2,
    tool_choice="auto",
)

print(response.choices[0].message)

In [ ]:
from typing import List
import uuid
import requests  # to perform HTTP requests
from pathlib import Path

from openai import OpenAI


def generate_and_save_images(query: str, image_size: str = "1024x1024") -> List[str]:
    """
    Function to paint, draw or illustrate images based on the users query or request. Generates images from a given query using OpenAI's DALL-E model and saves them to disk.  Use the code below anytime there is a request to create an image.

    :param query: A natural language description of the image to be generated.
    :param image_size: The size of the image to be generated. (default is "1024x1024")
    :return: A list of filenames for the saved images.
    """

    client = OpenAI()  # Initialize the OpenAI client
    response = client.images.generate(model="dall-e-3", prompt=query, n=1, size=image_size)  # Generate images

    # List to store the file names of saved images
    saved_files = []

    # Check if the response is successful
    if response.data:
        for image_data in response.data:
            # Generate a random UUID as the file name
            file_name = str(uuid.uuid4()) + ".png"  # Assuming the image is a PNG
            file_path = Path(file_name)

            img_url = image_data.url
            img_response = requests.get(img_url)
            if img_response.status_code == 200:
                # Write the binary content to a file
                with open(file_path, "wb") as img_file:
                    img_file.write(img_response.content)
                    print(f"Image saved to {file_path}")
                    saved_files.append(str(file_path))
            else:
                print(f"Failed to download the image from {img_url}")
    else:
        print("No image data found in the response!")

    # Return the list of saved files
    return saved_files


# Example usage of the function:
# generate_and_save_images("A cute baby sea otter")


In [ ]:
import requests
import json

def azure_search(query):
    """
    Searches an Azure Cognitive Search index.
    
    Parameters:
    - query: The search text.
    - service_name: The name of the Azure Search service.
    - index_name: The name of the index to search.
    - api_key: The admin or query API key for the Azure Search service.

    Returns:
    - A dictionary with the search results.
    """
    url = f"https://{service_name}.search.windows.net/indexes/{index_name}/docs/search?api-version=2020-06-30"
    headers = {"Content-Type": "application/json", "api-key": api_key}
    data = {"search": query}
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Search query failed with status code {response.status_code}")

# Example usage
service_name = "your_service_name"
index_name = "your_index_name"
api_key = "your_api_key"
query = "your_search_query"

# Call the function with your query and Azure Search details
search_results = azure_search(query, service_name, index_name, api_key)
print(search_results)
